In [1]:
import os
import json
import anthropic
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

# Now you can access variables using os.getenv()
api_key = os.getenv("CLAUDE_API_KEY")
  # For debugging purposes, remove in production

In [4]:
# Initialize the Anthropic client
client = anthropic.Anthropic(api_key=api_key)

# Folder containing batch files
BATCH_FOLDER = "batch_files"
BATCH_LOG_FILE = "batch_responses_log.jsonl"

In [5]:
batch_files = [f for f in os.listdir(BATCH_FOLDER) if f.endswith(".jsonl")]

# Prompt user to select one file
print("Available batch files:")
for i, file in enumerate(batch_files):
    print(f"{i + 1}. {file}")

Available batch files:
1. choice_prompts_with_norm.jsonl
2. choice_prompts_without_norm.jsonl
3. immoral_act_with_norm_prompts.jsonl
4. immoral_act_without_norm_prompts.jsonl
5. with_moralAction_immoralConsequnece_base_prompts.jsonl
6. with_immoralAction_moralConsequnece_base_prompts.jsonl
7. injection_moralAction_immoralOutcome_prompts.jsonl
8. anti_action_immoralAction_prompts.jsonl
9. pro_outcome_immoralAction_prompts.jsonl
10. outcome_weighted_moralAction_prompts.jsonl


In [8]:
def process_batch_file(filepath):
    if not os.path.exists(filepath):
        print(f"❌ File not found: {filepath}")
        return

    filename = os.path.basename(filepath)
    requests_to_send = []

    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                try:
                    req = json.loads(line)
                    if "custom_id" in req and "params" in req:
                        requests_to_send.append(req)
                    else:
                        print(f"⚠️ Skipping malformed entry in {filename}: {line.strip()}")
                except Exception as e:
                    print(f"❌ Error parsing line in {filename}: {e}")

    print(f"✅ Loaded {len(requests_to_send)} requests from {filename}")

    if not requests_to_send:
        print("⚠️ No valid requests found to send.")
        return

    try:
        message_batch = client.messages.batches.create(requests=requests_to_send)
        print("🎯 Batch Created:")
        print(message_batch)

        # Append the batch response to a log file
        with open(BATCH_LOG_FILE, "a", encoding="utf-8") as log_file:
            log_file.write(json.dumps({
                "batch_file": filename,
                "response": message_batch.model_dump()
            }, default=str) + "\n")
            print(f"📦 Batch response logged in: {BATCH_LOG_FILE}")

    except Exception as e:
        print(f"❌ Error sending batch: {e}")

In [ ]:

batch_file_path = "batch_files/with_moralAction_immoralConsequnece_base_prompts.jsonl"  # replace with your file
process_batch_file(batch_file_path)

❌ File not found: batch_files/with_moralAction_moralConsequnece_base_prompts.jsonl


In [8]:
import json
from datetime import datetime, timezone

# Manually entered batch metadata (based on your printout)
batch_info = {
    "batch_file": "anti_action_immoralAction_prompts.jsonl",
    "response": {
        "id": "msgbatch_01NjacNb4xK6STxoTuAXkgzE",
        "archived_at": None,
        "cancel_initiated_at": None,
        "created_at": datetime(2025, 5, 16, 17, 14, 56, 797993, tzinfo=timezone.utc).isoformat(),
        "ended_at": None,
        "expires_at": datetime(2025, 5, 17, 17, 14, 56, 797993, tzinfo=timezone.utc).isoformat(),
        "processing_status": "in_progress",
        "request_counts": {
            "canceled": 0,
            "errored": 0,
            "expired": 0,
            "processing": 12000,
            "succeeded": 0
        },
        "results_url": None,
        "type": "message_batch"
    }
}

# Path to your log file
BATCH_LOG_FILE = "batch_responses_log.jsonl"

# Append the manually structured entry
with open(BATCH_LOG_FILE, "a", encoding="utf-8") as log_file:
    log_file.write(json.dumps(batch_info) + "\n")

print(f"📦 Manually logged batch: {batch_info['response']['id']}")


📦 Manually logged batch: msgbatch_01NjacNb4xK6STxoTuAXkgzE
